In [121]:
import pandas as pd
import numpy as np
import re

# 1.Reading in the Data:

In [5]:
df_win = pd.read_pickle("../Data/Match_Info/match_winner_data.pickle")

In [6]:
df_lose = pd.read_pickle("../Data/Match_Info/match_loser_data.pickle")

In [19]:
df_match = pd.read_pickle("../Data/Match_Info/match_data.pickle")

# 2. How to access the json dictionary lists:
first check out how to get the summonerName, then figure out how to get the information on the participants.

On the way start filtering only Solo-Q ranked games, which have the **queueId** 420 and the **gameMode** CLASSIC.

In [39]:
df_match=df_match[df_match["gameMode"]=="CLASSIC"]

In [54]:
df_match = df_match[df_match["queueId"]==420]

In [73]:
df_match = df_match.reset_index(drop=True)

### Looking into the participants:
The participants is structured as follows:

1. List of dictionaries
2. Contains 10 players for each game
3. each player gets some further information
4. **stats** and **timeline** are nested dictionaries within the dictionary

In order to find out, which of those to keep, lets take a deeper look into one of the participants and the information conntained:
        

In [56]:
list(df_match["participants"][1][0])

['participantId',
 'teamId',
 'championId',
 'spell1Id',
 'spell2Id',
 'stats',
 'timeline']

In [57]:
list(df_match["participants"][1][0]["stats"])

['participantId',
 'win',
 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'kills',
 'deaths',
 'assists',
 'largestKillingSpree',
 'largestMultiKill',
 'killingSprees',
 'longestTimeSpentLiving',
 'doubleKills',
 'tripleKills',
 'quadraKills',
 'pentaKills',
 'unrealKills',
 'totalDamageDealt',
 'magicDamageDealt',
 'physicalDamageDealt',
 'trueDamageDealt',
 'largestCriticalStrike',
 'totalDamageDealtToChampions',
 'magicDamageDealtToChampions',
 'physicalDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'totalHeal',
 'totalUnitsHealed',
 'damageSelfMitigated',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'visionScore',
 'timeCCingOthers',
 'totalDamageTaken',
 'magicalDamageTaken',
 'physicalDamageTaken',
 'trueDamageTaken',
 'goldEarned',
 'goldSpent',
 'turretKills',
 'inhibitorKills',
 'totalMinionsKilled',
 'neutralMinionsKilled',
 'neutralMinionsKilledTeamJungle',
 'neutralMinionsKilledEnemyJungle',
 'totalTimeCrowdControlDealt',
 'champLev

In [58]:
list(df_match["participants"][1][0]["timeline"])

['participantId',
 'creepsPerMinDeltas',
 'xpPerMinDeltas',
 'goldPerMinDeltas',
 'damageTakenPerMinDeltas',
 'role',
 'lane']

Of those entries I am picking a few to keep and drop the rest.

In [59]:
def parse_participant(participant_dict):
    keys_to_keep = ["participantId",'teamId','championId','spell1Id','spell2Id']
    stats_keys = (['win',"item0", "item1","item2","item3","item4","item5","item6", "kills", "deaths", "assists",
                  "doubleKills", "tripleKills", "quadraKills", "pentaKills", "goldEarned","role", "lane"])
    e = {}
    for key, value in participant_dict.items():
        if key in keys_to_keep:
            e[key] = value
        elif type(value) == dict:
            for key2, value2 in value.items():
                if key2 in stats_keys:
                    e[key2] = value2
                    
    return e

In [75]:
parts = df_match["participants"].copy()

In [76]:
for i in range(len(parts)):
    e = [parse_participant(part) for part in parts.loc[i]]
    parts[i] = e

In [97]:
df_match["participants"]=parts

The same is done for the **participantIdentities** column, although I just want to keep the summoner name, in case I want to look at certain summoners and their performance.

In [60]:
def parse_identity(identity):
    e = {}
    keeping_this = ["summonerName"]
    for key,value in identity.items():
        if type(value)==dict:
            for key2,value2 in value.items():
                if key2 in keeping_this:
                    e[key2]=value2
    return e

In [90]:
ident = df_match["participantIdentities"].copy()

In [91]:
for i in range(len(df_match)):
    ident[i] = [parse_identity(part) for part in ident.loc[i]]

In [98]:
df_match["participantIdentities"]=ident

# 3. Dropping unnessesary columns:

In [109]:
df_match = df_match.drop(columns=["status.status_code","status.message", "seasonId", "gameType", "gameMode", "mapId", "platformId"])

In [110]:
df_match.head()

,gameCreation,gameDuration,gameId,gameVersion,participantIdentities,participants,queueId
0,1.585155e+12,1323.0,4.247263e+09,10.6.314.4405,"[{'summonerName': '쪼렙이다말로하자'}, {'summonerName'...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
1,1.585152e+12,1317.0,4.247156e+09,10.6.314.4405,"[{'summonerName': 'LPL Player'}, {'summonerNam...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
2,1.585059e+12,932.0,4.243963e+09,10.6.313.8894,"[{'summonerName': '호잇이'}, {'summonerName': 'Ge...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
3,1.584978e+12,2098.0,4.241678e+09,10.6.313.8894,"[{'summonerName': '213321123'}, {'summonerName...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0
4,1.584973e+12,2344.0,4.241539e+09,10.6.313.8894,"[{'summonerName': 'ggffggg'}, {'summonerName':...","[{'participantId': 1, 'teamId': 100, 'champion...",420.0


# 4. Changing the version to just show the patch:
Removing the tails of the patch numbers: 10.6.314.4405 --> 10.6

In [115]:
for i in range(len(df_match)):
    df_match["gameVersion"][i] = df_match["gameVersion"][i][:4]

/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [129]:
for i in range(len(df_match)):
    df_match["gameVersion"][i]= df_match["gameVersion"][i].strip(".")

/Users/andreasglassl/opt/miniconda3/envs/backpack_banana/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [136]:
df_match["gameVersion"]=df_match["gameVersion"].astype(float)

In [138]:
df_match.to_pickle("../Data/Match_Info/match_data.pickle")